In [9]:
import DataPipeline
#import model
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np

In [4]:
# Imputer
df = pd.read_csv("../data/Kagg_life_data.csv")

df_floats = df.drop(columns=["Country", "Status"])
print(df_floats.isna().sum())

imputer = KNNImputer().fit(df_floats)

imputed_df = imputer.transform(df_floats)

df[df_floats.columns] = imputed_df
print(df.isna().sum())

Year                                 0
Life expectancy                      0
Adult Mortality                      0
infant deaths                        0
Alcohol                            188
percentage expenditure               0
Hepatitis B                        536
Measles                              0
 BMI                                32
under-five deaths                    0
Polio                               19
Total expenditure                  221
Diphtheria                          19
 HIV/AIDS                            0
GDP                                411
Population                         612
 thinness  1-19 years               32
 thinness 5-9 years                 32
Income composition of resources    144
Schooling                          144
dtype: int64
Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths         

In [13]:
# Isolation Forest to remove outliers

def remove_outliers(self, data: pd.DataFrame, cutoff: float = 0.0) -> pd.DataFrame:
    """Remove outliers with an isolation forest
    
    Only numerical data is modified. All NaNs must already be removed.

    Parameters:
    ----------
        data  (pd.DataFrame): dataframe to alter
        cutoff (float): Cutoff for anomaly scores. Typically, anything less than 0 
            is considered an outlier


    """
    # Record original n_samples
    n_samples = data.shape[0]

    # Change only numeric data
    numeric_cols = data.select_dtypes(include=['float64', 'int']).columns.tolist()
    numeric_data = data[numeric_cols]
    
    # Create anomaly scores for dataframe
    forest = IsolationForest().fit(numeric_data)
    scores = forest.decision_function(numeric_data)

    # Filter out anomolous rows
    data = data[scores >= cutoff]

    # Print % of data removed
    print( (scores < cutoff).sum() / n_samples)

    return data


# Change only numeric data
numeric_cols = df.select_dtypes(include=['float64', 'int']).columns.tolist()
numeric_data = df[numeric_cols]

# Create anomaly scores for dataframe
forest = IsolationForest().fit(numeric_data)
scores = forest.decision_function(numeric_data)
print((scores < 0).sum())


# Filter out anomolous rows
data = df[scores >= -0.01]

print(data.shape, df.shape)
print(data.head())

219
(2677, 22) (2848, 22)
       Country    Year      Status  Life expectancy   Adult Mortality  \
1  Afghanistan  2014.0  Developing              59.9            271.0   
2  Afghanistan  2013.0  Developing              59.9            268.0   
3  Afghanistan  2012.0  Developing              59.5            272.0   
4  Afghanistan  2011.0  Developing              59.2            275.0   
5  Afghanistan  2010.0  Developing              58.8            279.0   

   infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   ...  \
1           64.0     0.01               73.523582         62.0     492.0  ...   
2           66.0     0.01               73.219243         64.0     430.0  ...   
3           69.0     0.01               78.184215         67.0    2787.0  ...   
4           71.0     0.01                7.097109         68.0    3013.0  ...   
5           74.0     0.01               79.679367         66.0    1989.0  ...   

   Polio  Total expenditure  Diphtheria    HIV/A

In [ ]:
def scale_standard(self, data: pd.DataFrame) -> pd.DataFrame:
    """Standardize features by making the mean 0 and the variance 1
    
    Only numerical data is modified. All NaNs must already be removed.

    Parameters:
    ----------
        data  (pd.DataFrame): dataframe to alter

    """# Change only numeric data
    numeric_cols = data.select_dtypes(include=['float64', 'int']).columns.tolist()
    numeric_data = data[numeric_cols]
    
    # Scale Data
    scaler = StandardScaler().fit(numeric_data)
    scaled_data = scaler.transform(numeric_data)

    # Scale original data
    data.loc[:, numeric_cols] = scaled_data

    return data

# Change only numeric data
numeric_cols = data.select_dtypes(include=['float64', 'int']).columns.tolist()
numeric_data = data[numeric_cols]

# Scale Data
scaler = StandardScaler().fit(numeric_data)
scaled_data = scaler.transform(numeric_data)

# Scale original data
data.loc[:, numeric_cols] = scaled_data

print(data.head())

       Country      Year      Status  Life expectancy   Adult Mortality  \
1  Afghanistan  1.399382  Developing         -1.106250         0.968508   
2  Afghanistan  1.180411  Developing         -1.106250         0.942456   
3  Afghanistan  0.961441  Developing         -1.150839         0.977192   
4  Afghanistan  0.742471  Developing         -1.184281         1.003243   
5  Afghanistan  0.523500  Developing         -1.228870         1.037979   

   infant deaths   Alcohol  percentage expenditure  Hepatitis B  Measles   \
1       1.584903 -1.164872               -0.350119    -0.944814 -0.142447   
2       1.649585 -1.164872               -0.350292    -0.853639 -0.156397   
3       1.746609 -1.164872               -0.347476    -0.716877  0.373958   
4       1.811291 -1.164872               -0.387785    -0.671290  0.424811   
5       1.908314 -1.164872               -0.346628    -0.762464  0.194398   

   ...     Polio  Total expenditure  Diphtheria    HIV/AIDS       GDP  \
1  ... -1.212